# CNN Network 

# Environment Init

In [27]:
import torch
import torch.nn.functional as F
import os
# import cv2 as cv
import numpy as np
from typing import Tuple
import torchvision
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
   
curPath=os.path.abspath('')
trainPath=curPath+"/../train"
print(torch.__version__)
print(torchvision.__version__)


1.13.1
0.13.1a0


# Create Train Set & Test Set

In [28]:
BATCH_SIZE=512 #大概需要2G的显存
EPOCHS=50 # 总共训练批次
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 让torch判断是否使用GPU，建议使用GPU环境，因为会快很多
transform = transforms.Compose([
    # transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.Grayscale(num_output_channels=1)
])

# Load the dataset using ImageFolder
dataset = ImageFolder(root=trainPath, transform=transform)
trainSet,testSet=torch.utils.data.random_split(dataset,[int(len(dataset)*0.9),len(dataset)-int(len(dataset)*0.9)])
print(len(trainSet))
print(len(testSet))
trainLoader=DataLoader(
    trainSet,
    batch_size=BATCH_SIZE,
    shuffle=True
 )
testLoader=DataLoader(
    testSet,
    batch_size=BATCH_SIZE,
    shuffle=True
)
for i,data in enumerate(trainLoader):
    print(i)

6696
744
0
1
2
3
4
5
6
7
8
9
10
11
12
13


# Net Structure

In [29]:
class ConvNet(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=12, kernel_size=5) 
        self.conv2 = torch.nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3) 
        
        self.ln1 = torch.nn.Linear(in_features=24*10*10, out_features=512)
        self.ln2 = torch.nn.Linear(in_features=512, out_features=12) 
    def forward(self,x):
        in_size = x.size(0) # batch_size=512
        out = self.conv1(x) # 1*28*28 -> 12*24*24 kernel_size=5
        out = torch.sigmoid(out) # 12*24*24
        out = F.max_pool2d(out, 2, 2) # 12*24*24 -> 12*12*12 pooling
        out = self.conv2(out) # 12*12*12 -> 24*10*10 kernel_size=3
        out = torch.sigmoid(out) # 24*10*10
        out = out.view(in_size, -1) # 24*10*10 -> 2400 Flatten
        out = self.ln1(out) # 2400 -> 512
        out = torch.sigmoid(out) # 512
        out = self.ln2(out) # 512 -> 12
        out = F.log_softmax(out, dim=1)
        return out

# Optimizer

In [30]:
model = ConvNet().to(DEVICE)
# Define the optimizer

optimizer = torch.optim.Adam(model.parameters())

# optimizer = torch.optim.SGD(model.parameters(), lr=0.05, momentum=0.9)

# Train and Test

In [31]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if(batch_idx+1)%5 == 0 or batch_idx==13: 
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # 将一批的损失相加
            pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

# Load Model

In [ ]:
# load the saved model
model.load_state_dict(torch.load('model.pth'))

# Main

In [32]:
for epoch in range(1, EPOCHS + 1):
    train(model, DEVICE, trainLoader, optimizer, epoch)
    test(model, DEVICE, testLoader)

Train Epoch: 1 [2048/6696 (29%)]	Loss: 2.557626
Train Epoch: 1 [4608/6696 (64%)]	Loss: 2.532242
Train Epoch: 1 [520/6696 (93%)]	Loss: 2.531694

Test set: Average loss: 2.5204, Accuracy: 65/744 (9%)

Train Epoch: 2 [2048/6696 (29%)]	Loss: 2.484224
Train Epoch: 2 [4608/6696 (64%)]	Loss: 2.496735
Train Epoch: 2 [520/6696 (93%)]	Loss: 2.445784

Test set: Average loss: 2.4817, Accuracy: 55/744 (7%)

Train Epoch: 3 [2048/6696 (29%)]	Loss: 2.490759
Train Epoch: 3 [4608/6696 (64%)]	Loss: 2.451891
Train Epoch: 3 [520/6696 (93%)]	Loss: 2.454117

Test set: Average loss: 2.4311, Accuracy: 164/744 (22%)

Train Epoch: 4 [2048/6696 (29%)]	Loss: 2.401308
Train Epoch: 4 [4608/6696 (64%)]	Loss: 2.341498
Train Epoch: 4 [520/6696 (93%)]	Loss: 2.212756

Test set: Average loss: 2.2205, Accuracy: 299/744 (40%)

Train Epoch: 5 [2048/6696 (29%)]	Loss: 2.138005
Train Epoch: 5 [4608/6696 (64%)]	Loss: 1.926183
Train Epoch: 5 [520/6696 (93%)]	Loss: 1.736587

Test set: Average loss: 1.6915, Accuracy: 477/744 (64%)


# Dump Model

In [34]:
# save the model
torch.save(model.state_dict(), 'cnnCharacterClassificaion.pt')